In [1]:
#import modules
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import resample


pd.options.display.max_columns=999
pd.options.display.max_rows=3000

from openpyxl.workbook import Workbook

In [2]:
data_file = pd.read_excel('Collections Case Study Data.xlsx', sheet_name=None)

## Model Data Cleaning 

In [3]:
data_file.keys()

dict_keys(['Loan Details', 'Loan_Status_MartoMay', 'Loan_Status_AprtoJun', 'Historical 6 Months Details', 'Loan_ID mapping', 'Call Details'])

In [5]:
df = data_file

In [6]:
#creates stratums for data based on month
mar_data = df['Loan_Status_MartoMay'].loc[df['Loan_Status_MartoMay']['Month'] == 'March']
apr_data = df['Loan_Status_MartoMay'].loc[df['Loan_Status_MartoMay']['Month'] == 'April']
may_data = df['Loan_Status_MartoMay'].loc[df['Loan_Status_MartoMay']['Month'] == 'May']
marapr_data = df['Loan_Status_AprtoJun'].loc[df['Loan_Status_AprtoJun']['Month'] == 'April']
aprmay_data = df['Loan_Status_AprtoJun'].loc[df['Loan_Status_AprtoJun']['Month'] == 'May']
mayjun_data = df['Loan_Status_AprtoJun'].loc[df['Loan_Status_AprtoJun']['Month'] == 'June']

In [7]:
#QC checking to see if our assumption is correct
print(len(mar_data) == len(marapr_data))
print(len(apr_data) == len(aprmay_data))
print(len(may_data) == len(mayjun_data))

True
True
True


In [8]:
#creates stratums for each month with performance in the following month
mar_data1 = pd.merge(mar_data, marapr_data, how='left', on='Loan_id', suffixes=('_mar', '_marapr'))
apr_data1 = pd.merge(apr_data, aprmay_data, how='left', on='Loan_id', suffixes=('_apr', '_aprmay'))
may_data1 = pd.merge(may_data, mayjun_data, how='left', on='Loan_id', suffixes=('_may', '_mayjun'))

In [9]:
#merges monthly data into one dataframe
data1 = pd.merge(mar_data1, apr_data1, how='outer', on='Loan_id')
data1 = pd.merge(data1, may_data1, how='outer', on='Loan_id')

In [10]:
print(mar_data1.shape)
print(apr_data1.shape)
print(may_data1.shape)
print(data1.shape)

(24575, 13)
(27079, 13)
(29115, 13)
(40339, 37)


In [11]:
data1.head()

,Loan_id,Bucket_mar,TENURE_mar,PRINBALANCE_mar,Months on Books_mar,Sanctioned Amount_mar,Month_mar,Bucket_marapr,PRINBALANCE_marapr,TENURE_marapr,Months on Books_marapr,Sanctioned Amount_marapr,Month_marapr,Bucket_apr,TENURE_apr,PRINBALANCE_apr,Months on Books_apr,Sanctioned Amount_apr,Month_apr,Bucket_aprmay,PRINBALANCE_aprmay,TENURE_aprmay,Months on Books_aprmay,Sanctioned Amount_aprmay,Month_aprmay,Bucket_may,TENURE_may,PRINBALANCE_may,Months on Books_may,Sanctioned Amount_may,Month_may,Bucket_mayjun,PRINBALANCE_mayjun,TENURE_mayjun,Months on Books_mayjun,Sanctioned Amount_mayjun,Month_mayjun
0,A205346000,TB0,36.0,3693500.0,33.0,22000000.0,March,TB1,1332100.0,36.0,34.0,22000000.0,April,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A675851816,TB0,48.0,24353700.0,30.0,40000000.0,March,TB1,20875500.0,48.0,31.0,40000000.0,April,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A260852240,TB0,48.0,42514900.0,24.0,60000000.0,March,TB1,37947922.5,48.0,25.0,60000000.0,April,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A374402177,TB0,36.0,12290300.0,19.0,20000000.0,March,TB0,11214900.0,36.0,20.0,20000000.0,April,TB0,36.0,11214900.0,20.0,20000000.0,April,TB0,10639500.0,36.0,21.0,20000000.0,May,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A211889361,TB0,36.0,9396700.0,25.0,22000000.0,March,TB0,8666200.0,36.0,26.0,22000000.0,April,TB0,36.0,8666200.0,26.0,22000000.0,April,TB1,8666200.0,36.0,27.0,22000000.0,May,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
data1.columns

Index(['Loan_id', 'Bucket_mar', 'TENURE_mar', 'PRINBALANCE_mar',
       'Months on Books_mar', 'Sanctioned Amount_mar', 'Month_mar',
       'Bucket_marapr', 'PRINBALANCE_marapr', 'TENURE_marapr',
       'Months on Books_marapr', 'Sanctioned Amount_marapr', 'Month_marapr',
       'Bucket_apr', 'TENURE_apr', 'PRINBALANCE_apr', 'Months on Books_apr',
       'Sanctioned Amount_apr', 'Month_apr', 'Bucket_aprmay',
       'PRINBALANCE_aprmay', 'TENURE_aprmay', 'Months on Books_aprmay',
       'Sanctioned Amount_aprmay', 'Month_aprmay', 'Bucket_may', 'TENURE_may',
       'PRINBALANCE_may', 'Months on Books_may', 'Sanctioned Amount_may',
       'Month_may', 'Bucket_mayjun', 'PRINBALANCE_mayjun', 'TENURE_mayjun',
       'Months on Books_mayjun', 'Sanctioned Amount_mayjun', 'Month_mayjun'],
      dtype='object')

In [12]:
data1 = data1.drop(columns=['Month_mar', 'TENURE_marapr', 'Sanctioned Amount_marapr', 'Month_marapr', 'TENURE_apr', 'Sanctioned Amount_apr','Month_apr',
                            'TENURE_apr31','Sanctioned Amount_apr31', 'Month_apr31',
                            'TENURE_may01', 'Sanctioned Amount_may01', 'Month_may01',
                            'TENURE_may31', 'Sanctioned Amount_may31', 'Month_may31'
                           ])

In [14]:
data1.describe()

,TENURE_mar01,PRINBALANCE_mar01,Months on Books_mar01,Sanctioned Amount_mar01,PRINBALANCE_mar31,Months on Books_mar31,PRINBALANCE_apr01,Months on Books_apr01,PRINBALANCE_apr31,Months on Books_apr31,PRINBALANCE_may01,Months on Books_may01,PRINBALANCE_may31,Months on Books_may31
count,24575.000000,2.457500e+04,24575.000000,2.457500e+04,2.457500e+04,24575.000000,2.707900e+04,27079.000000,2.707900e+04,27079.000000,2.911500e+04,29115.000000,2.911500e+04,29115.000000
mean,41.832838,3.368172e+07,15.600895,4.603559e+07,3.183383e+07,16.600895,3.340031e+07,15.773847,3.157736e+07,16.773847,3.415342e+07,15.400996,3.231523e+07,16.400996
std,7.599259,2.397062e+07,10.692061,2.586228e+07,2.367301e+07,10.692061,2.364095e+07,10.643053,2.335223e+07,10.643053,2.497425e+07,10.549727,2.467654e+07,10.549727
min,12.000000,4.090000e+05,0.000000,1.000000e+07,2.000000e+02,1.000000,7.470000e+04,0.000000,1.000000e+03,1.000000,2.372000e+05,0.000000,2.000000e+02,1.000000
25%,36.000000,1.762650e+07,6.000000,3.000000e+07,1.597412e+07,7.000000,1.757270e+07,7.000000,1.592710e+07,8.000000,1.807675e+07,6.000000,1.641180e+07,7.000000
50%,48.000000,2.906860e+07,14.000000,4.000000e+07,2.757780e+07,15.000000,2.893120e+07,15.000000,2.740140e+07,16.000000,2.940660e+07,14.000000,2.798320e+07,15.000000
75%,48.000000,4.406620e+07,23.000000,5.500000e+07,4.230300e+07,24.000000,4.370955e+07,23.000000,4.205315e+07,24.000000,4.434225e+07,22.000000,4.268020e+07,23.000000
max,239.000000,3.753802e+08,74.000000,4.200000e+08,3.732658e+08,75.000000,3.743314e+08,78.000000,3.721831e+08,79.000000,1.137036e+09,84.000000,1.131137e+09,85.000000


In [32]:
data2=data1

In [33]:

data2['RF1'] = np.where(data2['Bucket_mar01']=='TB0', 1, 0)
data2['RF2'] = np.where((data2['Bucket_apr01']=='TB0')|(data2['Bucket_apr01']=='TB1'), 1, 0)
data2['RF3'] = np.where((data2['Bucket_may01']=='TB0')|(data2['Bucket_may01']=='TB1'), 1, 0)
data2['RF4'] = np.where((data2['Bucket_may31']=='TB0')|(data2['Bucket_may31']=='TB1'), 1, 0)

In [34]:
data2['RB2'] = np.where((data2['Bucket_apr01']=='TB0')&(data2['Bucket_mar01']=='TB0'), 1, 0)
data2['RB3'] = np.where(((data2['Bucket_may01']=='Nan')&(data2['Bucket_apr01']=='TB0')
                         |(data2['Bucket_may01']=='REGULAR')&(data2['Bucket_apr01']=='TB0')
                        |(data2['Bucket_may01']=='TB0')&(data2['Bucket_apr01']=='TB1')
                        ), 1, 0)
data2['RB4'] = np.where(((data2['Bucket_may31']=='Nan')&(data2['Bucket_may01']=='TB0')
                         |(data2['Bucket_may31']=='REGULAR')&(data2['Bucket_may01']=='TB0')
                        |(data2['Bucket_may31']=='TB0')&(data2['Bucket_may01']=='TB1')
                        ), 1, 0)

In [35]:
data2['RF4'].value_counts()

0    38309
1     2030
Name: RF4, dtype: int64

In [189]:
data2.head()

,Loan_id,Bucket_mar01,TENURE_mar01,PRINBALANCE_mar01,Months on Books_mar01,Sanctioned Amount_mar01,Bucket_mar31,PRINBALANCE_mar31,Months on Books_mar31,Bucket_apr01,PRINBALANCE_apr01,Months on Books_apr01,Bucket_apr31,PRINBALANCE_apr31,Months on Books_apr31,Bucket_may01,PRINBALANCE_may01,Months on Books_may01,Bucket_may31,PRINBALANCE_may31,Months on Books_may31,RF1,RF2,RF3,RF4,RB2,RB3,RB4
0,A205346000,TB0,36.0,3693500.0,33.0,22000000.0,TB1,1332100.0,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0
1,A675851816,TB0,48.0,24353700.0,30.0,40000000.0,TB1,20875500.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0
2,A260852240,TB0,48.0,42514900.0,24.0,60000000.0,TB1,37947922.5,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0,0
3,A374402177,TB0,36.0,12290300.0,19.0,20000000.0,TB0,11214900.0,20.0,TB0,11214900.0,20.0,TB0,10639500.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0,1,0,0
4,A211889361,TB0,36.0,9396700.0,25.0,22000000.0,TB0,8666200.0,26.0,TB0,8666200.0,26.0,TB1,8666200.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0,0,1,0,0


In [37]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40339 entries, 0 to 40338
Data columns (total 28 columns):
Loan_id                    40339 non-null object
Bucket_mar01               24575 non-null object
TENURE_mar01               24575 non-null float64
PRINBALANCE_mar01          24575 non-null float64
Months on Books_mar01      24575 non-null float64
Sanctioned Amount_mar01    24575 non-null float64
Bucket_mar31               24575 non-null object
PRINBALANCE_mar31          24575 non-null float64
Months on Books_mar31      24575 non-null float64
Bucket_apr01               27079 non-null object
PRINBALANCE_apr01          27079 non-null float64
Months on Books_apr01      27079 non-null float64
Bucket_apr31               27079 non-null object
PRINBALANCE_apr31          27079 non-null float64
Months on Books_apr31      27079 non-null float64
Bucket_may01               29115 non-null object
PRINBALANCE_may01          29115 non-null float64
Months on Books_may01      29115 non-null float

In [170]:
data2.columns

Index(['Loan_id', 'Bucket_mar01', 'TENURE_mar01', 'PRINBALANCE_mar01',
       'Months on Books_mar01', 'Sanctioned Amount_mar01', 'Bucket_mar31',
       'PRINBALANCE_mar31', 'Months on Books_mar31', 'Bucket_apr01',
       'PRINBALANCE_apr01', 'Months on Books_apr01', 'Bucket_apr31',
       'PRINBALANCE_apr31', 'Months on Books_apr31', 'Bucket_may01',
       'PRINBALANCE_may01', 'Months on Books_may01', 'Bucket_may31',
       'PRINBALANCE_may31', 'Months on Books_may31', 'RF1', 'RF2', 'RF3',
       'RF4', 'RB2', 'RB3', 'RB4'],
      dtype='object')

### TB0 Data 

In [107]:
data_TB0 = pd.DataFrame(df['Loan_Status_MartoMay'])
data_TB0 = data_TB0.drop_duplicates(subset ="Loan_id", keep = 'first', inplace = False) 
data_TB0 = data_TB0.drop(columns=['TENURE', 'Sanctioned Amount','Month',  'Bucket'])
data_TB0.info() 
                               

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40339 entries, 0 to 80759
Data columns (total 3 columns):
Loan_id            40339 non-null object
PRINBALANCE        40339 non-null float64
Months on Books    40339 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.2+ MB


### Historical Data Prep

In [74]:
#cleaning/processing 6 month data 
data_6m = pd.DataFrame(df['Historical 6 Months Details'])
data_6m = data_6m.drop_duplicates(subset ="Loan_id", keep = 'first', inplace = False) 
data_6m = data_6m.drop(columns=['rollf', 'Month'])
data_6m.info()

### Call Data Prep

In [243]:
#assigns 'Call Detail' table to dataframe
call_data = df['Call Details']
call_data.tail()

,month,Application_Id,Login_ID,Right_Party_Contact,Promise_to_pay,total_contacts
79609,5,MX-999897229,AGNT19,0,0,2
79610,5,MX-999911711,AGNT6,0,0,1
79611,5,MX-999978990,AGNT43,2,0,2
79612,5,MX-999980626,AGNT21,1,1,1
79613,5,MX-999985403,AGNT19,5,1,6


In [244]:
#groups call data by id and then aggregates
month_table = call_data.groupby(['Application_Id']).aggregate({'month':'nunique', 'Right_Party_Contact':'sum', 'Promise_to_pay':'sum', 'total_contacts':'sum'})

#calculates Right Party Contacts (RPC) rate and Percentage of outbound calls resulting in Promise to Pay (PTP)
# month_table['avg_intensity'] = round(month_table['total_contacts'] / month_table['month'], 3)
month_table['RPC_rate'] = round(month_table['Right_Party_Contact'] / month_table['total_contacts'], 3)
month_table['PTP_rate'] = round(month_table['Promise_to_pay'] / month_table['total_contacts'], 3)
month_table.rename(columns={'month':'month_count'}, inplace=True)



In [245]:
call_data = pd.merge(df['Loan_ID mapping'], month_table, how='outer', left_on='Application_id', right_on='Application_Id')

In [246]:
call_data.fillna(0, inplace=True)
valid = len(month_table)
missing = len(call_data[call_data['total_contacts'].isnull()])
call_data[call_data['total_contacts'].isnull()]

,Loanid,Application_id,month_count,Right_Party_Contact,Promise_to_pay,total_contacts,RPC_rate,PTP_rate


In [247]:

call_data.rename(columns = {call_data.columns[0]: 'Loan_id'}, inplace = True)

In [248]:
call_data.head()

,Loan_id,Application_id,month_count,Right_Party_Contact,Promise_to_pay,total_contacts,RPC_rate,PTP_rate
0,A987633031,MX-100040250,1.0,5.0,2.0,21.0,0.238,0.095
1,A145720684,MX-10005082,1.0,0.0,0.0,2.0,0.000,0.000
2,A730581847,MX-100062359,1.0,1.0,1.0,1.0,1.000,1.000
3,A134412216,MX-100095892,1.0,1.0,0.0,2.0,0.500,0.000
4,A897161619,MX-100116696,3.0,5.0,2.0,10.0,0.500,0.200


## Target Prep

In [178]:
target_prep = data2.drop(columns=['Bucket_mar01', 'TENURE_mar01', 'PRINBALANCE_mar01',
       'Months on Books_mar01', 'Sanctioned Amount_mar01', 'Bucket_mar31',
       'PRINBALANCE_mar31', 'Months on Books_mar31', 'Bucket_apr01',
       'PRINBALANCE_apr01', 'Months on Books_apr01', 'Bucket_apr31',
       'PRINBALANCE_apr31', 'Months on Books_apr31', 'Bucket_may01',
       'PRINBALANCE_may01', 'Months on Books_may01', 'Bucket_may31',
       'PRINBALANCE_may31', 'Months on Books_may31',
                               ])

In [179]:
target_prep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40339 entries, 0 to 40338
Data columns (total 8 columns):
Loan_id    40339 non-null object
RF1        40339 non-null int32
RF2        40339 non-null int32
RF3        40339 non-null int32
RF4        40339 non-null int32
RB2        40339 non-null int32
RB3        40339 non-null int32
RB4        40339 non-null int32
dtypes: int32(7), object(1)
memory usage: 1.7+ MB


In [190]:

target_prep['RF_sum'] = target_prep['RF1'] + target_prep['RF2'] +target_prep['RF3'] + target_prep['RF4']
target_prep['target'] = target_prep['RF_sum'].apply(lambda x: 1 if x >1 else 0)
target_prep.columns

Index(['Loan_id', 'RF1', 'RF2', 'RF3', 'RF4', 'RB2', 'RB3', 'RB4', 'RF_sum',
       'target'],
      dtype='object')

In [192]:
target = target_prep.drop(columns=[ 'RF1', 'RF2', 'RF3', 'RF4', 'RB2', 'RB3', 'RB4', 'RF_sum'])

In [193]:
target.head()

,Loan_id,target
0,A205346000,0
1,A675851816,0
2,A260852240,0
3,A374402177,1
4,A211889361,1


### Features Combined

In [259]:
dataframe = pd.merge(data_TB0, df['Loan Details'], how='left', on='Loan_id')


In [260]:
dataframe = pd.merge(dataframe, data_6m, how='left', on='Loan_id')

In [261]:
dataframe = pd.merge(dataframe, call_data, how='left', on='Loan_id')

In [262]:
dataframe = pd.merge(dataframe, target, how='left', on='Loan_id')

In [263]:
dataframe = dataframe.drop(columns=[ 'Loan_id','Application_id','rollb'])

In [264]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40339 entries, 0 to 40338
Data columns (total 22 columns):
PRINBALANCE             40339 non-null float64
Months on Books         40339 non-null int64
Debt_to_burden_Ratio    40339 non-null float64
total_income            40339 non-null float64
TENURE                  40339 non-null int64
Sanctioned Amount       40339 non-null int64
paidcure                40339 non-null int64
paiduncure              40339 non-null int64
unpaid                  40339 non-null int64
num6mosdel              40339 non-null int64
num3mosdel              40339 non-null int64
num6mosdel_2plus        40339 non-null int64
num3mosdel_2plus        40339 non-null int64
max6del                 40339 non-null int64
max3del                 40339 non-null int64
month_count             40339 non-null float64
Right_Party_Contact     40339 non-null float64
Promise_to_pay          40339 non-null float64
total_contacts          40339 non-null float64
RPC_rate              

In [235]:
dataframe.head()

,PRINBALANCE,Months on Books,Debt_to_burden_Ratio,total_income,TENURE,Sanctioned Amount,paidcure,paiduncure,unpaid,num6mosdel,num3mosdel,num6mosdel_2plus,num3mosdel_2plus,max6del,max3del,month_count,Right_Party_Contact,Promise_to_pay,total_contacts,RPC_rate,PTP_rate,target
0,10539100.0,37,0.244840,5000000.0,48,30000000,2,1,3,5,2,4,2,5,5,NaN,NaN,NaN,NaN,NaN,NaN,1
1,3693500.0,33,0.247458,4199908.0,36,22000000,2,1,3,6,3,6,3,4,4,NaN,NaN,NaN,NaN,NaN,NaN,0
2,24353700.0,30,0.149219,9887482.0,48,40000000,2,1,3,4,3,3,3,4,4,NaN,NaN,NaN,NaN,NaN,NaN,0
3,4441700.0,30,0.178700,5000000.0,36,20000000,2,1,3,4,2,3,2,4,4,NaN,NaN,NaN,NaN,NaN,NaN,0
4,20002300.0,19,0.212669,6105249.0,36,30000000,2,2,2,5,3,2,2,3,3,NaN,NaN,NaN,NaN,NaN,NaN,0


## Model

In [217]:
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [265]:
X= dataframe.drop(columns=['target'])
y= dataframe['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [266]:
import time
xgb = XGBClassifier(n_estimators=100)
training_start = time.perf_counter()
xgb.fit(X_train, y_train)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
preds = xgb.predict(X_test)
prediction_end = time.perf_counter()
acc_xgb = (preds == y_test).sum().astype(float) / len(preds)*100
xgb_train_time = training_end-training_start
xgb_prediction_time = prediction_end-prediction_start
XGB_Accuracy = accuracy_score(y_test, preds)
XGB_AUC = roc_auc_score(y_test, preds)

print("XGBoost's prediction accuracy is: %3.2f" % (acc_xgb))
print("Time consumed for training: %4.3f" % (xgb_train_time))
print("Time consumed for prediction: %6.5f seconds" % (xgb_prediction_time)) 
print("AUC: " + str(XGB_AUC))


XGBoost's prediction accuracy is: 94.11
Time consumed for training: 3.438
Time consumed for prediction: 0.03862 seconds
AUC: 0.9503212141088636


In [267]:
RFC_Model = RandomForestClassifier()
RFC_Model.fit(X_train, y_train)
RFC_Predict = RFC_Model.predict(X_test)
RFC_Accuracy = accuracy_score(y_test, RFC_Predict)
print("Accuracy: " + str(RFC_Accuracy))

RFC_AUC = roc_auc_score(y_test, RFC_Predict) 
print("AUC: " + str(RFC_AUC))

Accuracy: 0.9425330528846154
AUC: 0.9522865847305401


In [268]:
#Logistic Regression
LR_Model = LogisticRegression()
training_start2 = time.perf_counter()
LR_Model.fit(X_train, y_train)
training_end2 = time.perf_counter()
prediction_start2 = time.perf_counter()
preds = LR_Model.predict(X_test)
prediction_end2 = time.perf_counter()
acc_LR = (preds == y_test).sum().astype(float) / len(preds)*100
LR_train_time = training_end2-training_start2
LR_prediction_time = prediction_end2-prediction_start2
LR_Accuracy = accuracy_score(y_test, preds)
LR_AUC = roc_auc_score(y_test, preds)

print("LR's prediction accuracy is: %3.2f" % (acc_LR))
print("Time consumed for training: %4.3f" % (LR_train_time))
print("Time consumed for prediction: %6.5f seconds" % (LR_prediction_time)) 
print("AUC: " + str(LR_AUC))

LR's prediction accuracy is: 64.55
Time consumed for training: 0.842
Time consumed for prediction: 0.00343 seconds
AUC: 0.5


In [270]:
print(xgb.feature_importances_)
print(RFC_Model.feature_importances_)


[0.00378651 0.00306394 0.0023139  0.00227875 0.00247637 0.00217809
 0.00227092 0.00239556 0.00298646 0.00252683 0.00793218 0.00315364
 0.00279523 0.         0.         0.9480186  0.00189721 0.00257675
 0.00211155 0.00263328 0.00260418]
[4.07018951e-02 3.49636705e-02 3.60901575e-02 3.38859482e-02
 6.12705471e-03 2.55934207e-02 1.17320577e-02 1.39745915e-03
 3.39903330e-03 4.69162439e-03 4.77917802e-03 5.31427496e-04
 3.95457553e-04 2.72424262e-03 2.04178210e-03 5.11288838e-01
 8.73855299e-02 4.48778303e-02 8.48535944e-02 3.34114667e-02
 2.91283313e-02]
